## Microsoft Entra ID の概要

Microsoft Entra ID（旧 Azure Active Directory）は、Microsoft のクラウドベースの ID およびアクセス管理サービスです。Microsoft 365、Azure、およびその他数千の SaaS アプリケーションの中央 ID プロバイダーとして機能します。

主な機能:
* **シングルサインオン（SSO）** - ユーザーは一度認証するだけで複数のアプリケーションにアクセス可能
* **多要素認証（MFA）** - 追加の検証方法によるセキュリティ強化
* **条件付きアクセス** - ユーザー、デバイス、場所、リスクに基づくポリシーベースのアクセス制御
* **アプリケーション統合** - OAuth 2.0、OpenID Connect、SAML などの最新の認証プロトコルをサポート

## 学習目標

Microsoft Entra ID は、AgentCore Identity の ID プロバイダーとして使用でき、ユーザーを認証し、ユーザーに代わって保護されたリソースにアクセスするようエージェントを認可させることができます。このノートブックでは、Inbound 認証に Entra ID を使用する方法を学びます。

- エージェントを呼び出す前にユーザーを認証する

## 認証コードフロー
OAuth 2.0 認証コードフローは、Web アプリケーションがユーザーを安全に認証し、アクセストークンを取得するための推奨アプローチです。

このフローには以下が含まれます：

1. ユーザーを認証のために Entra ID にリダイレクト
2. ログイン成功後に認証コードを受信
3. コードをアクセストークンとリフレッシュトークンに交換
4. トークンを使用して保護されたリソースにアクセス

この統合パターンにより、AgentCore はアプリケーションの安全で標準ベースの認証を維持しながら、Entra ID の堅牢な ID 管理機能を活用できます。

## 学習目標 1: AgentCore Identity で使用するための Entra ID のセットアップ

### ステップ 1: Entra ID テナントのセットアップ

Entra ID テナントは、組織を表す Microsoft Entra ID の専用インスタンスです。Microsoft のクラウド内にある組織の独立したディレクトリと考えてください。

主な特徴:

* **一意の ID** - 各テナントは一意のドメインを持ちます（例：yourcompany.onmicrosoft.com）
* **分離された境界** - あるテナント内のユーザー、グループ、アプリケーションは他のテナントから分離されています
* **管理制御** - テナント管理者がユーザー、セキュリティポリシー、アプリケーション登録を管理します
* **マルチドメインサポート** - デフォルトの .onmicrosoft.com ドメインとカスタムドメインの両方を含めることができます

実際の運用:

OAuth 2.0 統合のために Entra ID にアプリケーションを登録する場合、特定のテナント内に登録することになります。そのテナントのユーザーは、組織の資格情報を使用してアプリケーションに認証できます。

AgentCore 統合に必要なもの:

* **テナント ID** - Entra ID インスタンスの一意の識別子
* **アプリケーション登録** - テナント内に登録されたアプリ
* **適切な権限** - アプリケーションに設定されたアクセス権

このテナントベースのモデルにより、認証と認可が組織のセキュリティ境界内に維持されます。

テナントの作成手順については https://learn.microsoft.com/en-us/entra/fundamentals/create-new-tenant を参照してください。

注意:
1. Microsoft Entra ID は AWS のサービスではありません。コストに関連する情報については、Microsoft Entra ID のドキュメントを参照してください。
2. 以下のステップで使用されているスクリーンショットは変更される可能性があります。Entra ID アプリケーションのセットアップに関する最新のガイダンスについては、Microsoft Entra ID のドキュメントを参照することをお勧めします。

### ステップ 2: アプリケーションのセットアップ

1. https://portal.azure.com にアクセスし、画面上部の検索バーで「Entra ID」を検索します
<img src="images/entraid.jpg" width="75%">

2. `管理` &rarr; `アプリの登録` に移動します
<img src="images/app.registration.png" width="75%">

3. `新規登録` をクリックし、詳細を入力します。必ずマルチテナントオプションを選択してください
<img src="images/app.registration.form.png" width="75%">

4. クライアントシークレットを作成します。AgentCore Identity で使用するために clientId とクライアントシークレットをコピーします。
<img src="images/gather.client.info.png" width="75%">

5. OAuth 用のスコープを作成します。API の公開 &rarr; `スコープの追加` に移動します。完全なスコープをコピーして保存します。
<img src="images/expose.api.png" width="75%">

6. `デバイスコードフロー` を有効にします。
<img src="images/device.code.flow.png" width="75%"/>

## 学習目標 2 - Inbound 認証に Entra ID を使用したシンプルなエージェントのセットアップ

#### 前提条件

* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* AWS リージョンを「us-west-2」または Bedrock AgentCore をサポートする任意のリージョンに設定。サポートされているリージョンについては https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agentcore-regions.html を参照してください。
* Docker、Finch、または Podman がインストールされていること

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet # in quite mode to reduce output to the console/notebook

In [ ]:
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

print(f"AWS Region: {region}")

In [ ]:
import boto3

sts = boto3.client("sts")
account_id = sts.get_caller_identity().get("Account")

print(f"AWS Account: {account_id}")

#### ノートブック全体で必要な主要情報の環境変数を設定

注意: audience は上記のステップの「アプリケーション ID URI」と同じになります

In [ ]:
import os

# REPLACE WITH YOUR client_id
os.environ["client_id"] = "REPLACE_ME"

# REPLACE WITH YOUR secret
os.environ["secret"] = "REPLACE_ME"

# REPLACE WITH YOUR scopes
os.environ["scopes"] = "REPLACE_ME"

# REPLACE WITH YOUR tenant_id
os.environ["tenant_id"] = "REPLACE_ME"

# REPLACE WITH YOUR audience
os.environ["audience"] = "REPLACE_ME"

#### エージェントコード
このノートブックの主な学習目標は EntraID を使用した Inbound 認証を学ぶことなので、エージェントはシンプルにしています

In [ ]:
%%writefile strands_wo_memory.py
import asyncio

from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent

app = BedrockAgentCoreApp()
agent = Agent()

class StreamingQueue:
    def __init__(self):
        self.finished = False
        self.queue = asyncio.Queue()
        
    async def put(self, item):
        await self.queue.put(item)

    async def finish(self):
        self.finished = True
        await self.queue.put(None)

    async def stream(self):
        while True:
            item = await self.queue.get()
            if item is None and self.finished:
                break
            yield item

queue = StreamingQueue()

async def agent_task(user_message: str):
    try:
        await queue.put("Agent execution begins....")
        
        response = agent(user_message)
        # ... process agent response here ...
        await queue.put(response.message)
    except Exception as e:
        await queue.put(f"Failed with error: {repr(e)}")
    finally:
        await queue.put("Agent excecution finished")
        await queue.finish()

@app.entrypoint
async def strands_agent_bedrock(payload, context):
    print("Context object is ....", context)
    prompt = payload.get("prompt", "hello")
    
    task = asyncio.create_task(agent_task(prompt))
    
    async def stream_with_task():
        async for item in queue.stream():
            yield item
        await task
    
    return stream_with_task()

if __name__ == "__main__":
    app.run()


#### Inbound 認証を強制する `authorizer_configuration` を使用して Runtime を設定します。
EntraID を使用した Inbound 認証には `customJWTAuthorizer` を使用します。discovery_url がテナント ID を使用してビルドされていることに注目してください

In [ ]:
import os

from bedrock_agentcore_starter_toolkit import Runtime


agentcore_runtime = Runtime()

discovery_url = f"https://login.microsoftonline.com/{os.environ['tenant_id']}/.well-known/openid-configuration"

response = agentcore_runtime.configure(
    entrypoint="strands_wo_memory.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_wo_memory_entra_inbound",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedAudience": os.environ["audience"].split(" "),
            # More JWT authorization can be added, refer to:
            # https://docs.aws.amazon.com/bedrock-agentcore-control/latest/APIReference/API_CustomJWTAuthorizerConfiguration.html
        }
    },
)

print(f"Runtime Agent: {response}")

## 学習目標 3 - MSAL を使用した認証とベアラートークンの取得
#### デバイス認証コードを使用した認証フローの開始
<img src="images/msal_code.png" width="75%">
<img src="images/msa_enter_code.png" width="75%">

#### ユーザー ID とパスワードを使用してログインします。または、すでにログインしている場合はユーザーを選択します。
<img src="images/msal_select_user.png" width="75%">
<img src="images/msal_confirm.png" width="75%">
<img src="images/msal_done.png" width="75%">


#### 完了すると、ノートブックに制御が戻り、ユーザーのベアラートークンが利用可能になります。以下のスクリーンショットのようになります。リンクに従い、次のセルを実行したときに取得したコードを使用してください。
<img src="images/msal_bearer_token_received.png" width="75%">

In [ ]:
import msal
import webbrowser

# Configuration details from your Entra ID app registration
CLIENT_ID = os.environ["client_id"]
TENANT_ID = os.environ["tenant_id"]
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPES = os.environ["scopes"].split(" ")


app = msal.PublicClientApplication(
    client_id=CLIENT_ID,
    authority=AUTHORITY,
)

# Attempt to acquire token silently from cache first
result = app.acquire_token_silent(SCOPES, account=None)

if not result:
    # If no token in cache, initiate interactive flow
    flow = app.initiate_device_flow(scopes=SCOPES)
    if "user_code" not in flow:
        raise ValueError("Failed to initiate device flow. No user_code found.")

    print(f"Device Code Flow: {flow['message']}")
    webbrowser.open(flow["verification_uri"])

    result = app.acquire_token_by_device_flow(flow)

if "access_token" in result:
    access_token = result["access_token"]
    print(f"Bearer Token Received: {access_token[:10]}...")
else:
    print("Failed to obtain a bearer token")
    print(f"\tError: {result.get('error')}")
    print(f"\tError Description: {result.get('error_description')}")
    print(f"\tCorrelation ID: {result.get('correlation_id')}")

    raise Exception(f"Failed to obtain bearer token: {str(result)}")


bearer_token_entra = result["access_token"]

## 学習目標 3 - 先ほど取得したベアラートークンを使用してエージェントをデプロイし、呼び出す

#### Runtime エージェントをデプロイ
local_build が有効になっているため、ローカルの Docker が実行されている必要があります。代替として、CloudBuild を使用するには `local_build=False` を設定できます。

In [ ]:
strands_wo_memory_launch_response = agentcore_runtime.launch(
    local_build=True,
    auto_update_on_conflict=True,
)

In [ ]:
import urllib.parse
import requests
import json
import uuid

if not strands_wo_memory_launch_response.agent_arn:
    raise Exception(
        "Missing Runtime Agent ARN. Verify that the Runtime Agent was created successfully in the previous step."
    )

escaped_agent_arn = urllib.parse.quote(
    strands_wo_memory_launch_response.agent_arn, safe=""
)
url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations?qualifier=DEFAULT"

session_id = str(uuid.uuid1())
headers = {
    "Authorization": f"Bearer {bearer_token_entra}",
    "Content-Type": "application/json",
    "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
    "X-Amzn-Trace-Id": f"entra_id_inbound_sample_{session_id}",
}

http_response = requests.post(
    url,
    data=json.dumps(
        {"prompt": "Hello! I am John Doe. I like brick oven pizza!", "user_id": "user1"}
    ),
    headers=headers,
)
http_response.raise_for_status()


print(f"Agent Response: {http_response.text}")

#### このセッションでの以前のインタラクションは agents.messages を通じて利用可能です。AgentCore Memory は使用されていません。新しいセッション ID を使用すると、エージェントは以前のインタラクションを思い出せません。

In [ ]:
http_response = requests.post(
    url, data=json.dumps({"prompt": "Who am I?", "user_id": "user1"}), headers=headers
)
http_response.raise_for_status()

print(f"Agent Response: {http_response.text}")

#### 代替として、AgentCore Runtime オブジェクトを使用してエージェントを呼び出すことができます。ベアラートークンと同じセッション ID を渡して、以前のセッションを続行します。

In [ ]:
invoke_response = agentcore_runtime.invoke(
    {"prompt": "Who am I?", "user_id": "user1"},
    bearer_token=bearer_token_entra,
    session_id=session_id,
)

print(f"Agent Invoke Response: {invoke_response}")

## まとめとクリーンアップ

このノートブックでは以下のことを学びました：

- OAuth 2.0 認証コードフローを提供するための Entra ID API とアプリケーションのセットアップ
- AgentCore Runtime を作成し、Entra ID を使用した Inbound 認証でエージェントをデプロイ
- 保護されたエージェントにアクセスするためのトークンを取得

#### 作成されたリソース

In [ ]:
print(f"Runtime Agent Arn: {strands_wo_memory_launch_response.agent_id}")

#### AgentCore Runtime エージェントを削除

In [ ]:
agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=strands_wo_memory_launch_response.agent_id
)

In [ ]:
import json
from jwt import JWT

jwt = JWT()

# Decode the token (without verification for inspection purposes only)
# For production, always verify the token's signature and claims
decoded_token = jwt.decode(bearer_token_entra, do_verify=False)

print(f"Decoded Bearer Token (for inspection): \n{json.dumps(decoded_token, indent=4)}")